In [1]:
#import section

import sys
sys.path.append('modules')

import json
import time
import imaplib
import email
from bs4 import BeautifulSoup
import mailfuncs
import dbfuncs


In [2]:
#initialisation


with open('gmail.creds') as f:
    data = json.load(f)
    gmailServer = "imap.gmail.com"
    gmailPort   = 993    
    gmailAccount = data['account']
    gmailPassword = data['password']
    
    
with open('db.creds') as f:
    data = json.load(f)
    dbHost = data['host']
    dbPort = data['port']
    dbDatabase = data['database']
    dbUser = data['user']
    dbPassword = data['password'] 

In [3]:
#connect to db

engine = dbfuncs.dbGetEngine(dbHost,dbPort,dbDatabase,dbUser,dbPassword)

In [51]:
#receive latest email id from db

connection = engine.connect()

rs = connection.execute("SELECT order_id FROM public.order ORDER BY message_id DESC LIMIT 1")

dblast_email_id = str(rs.fetchone())


connection.close()

None


In [19]:
#disconnect from db

engine.dispose()

None


In [57]:
#connect to gmail

mail = imaplib.IMAP4_SSL(gmailServer)
mail.login(gmailAccount,gmailPassword)

('OK', [b'fitosystems@gmail.com authenticated (Success)'])

In [60]:
#querying and defining messages for processing


shiftFromLastMessage = 10

mail.select('fitosystems.ru')
typ, data = mail.search(None, 'ALL')
mail_ids = data[0]
id_list = list(map(lambda x: x.decode("utf-8"), mail_ids.split()))
first_email_id = id_list[0]
last_email_id = id_list[-1 - shiftFromLastMessage]


indexFrom = 0
indexTo = 0

try:
    indexFrom = id_list.index(dblatest_email_id) + 1
except ValueError:
    indexFrom = 0
    
indexTo = len(id_list) - 1 - shiftFromLastMessage


print(indexFrom)
print(indexTo)

12600
12675


In [ ]:
#processing instance

message_id = latest_email_id


typ, data = mail.fetch(str(message_id), '(RFC822)' ) 


rawEmail = data[0][1]
msg = email.message_from_bytes(rawEmail)


emailDate =  msg["Date"]
emailFrom =  mailfuncs.decodeHeaderValue(msg["From"])
emailSubject = mailfuncs.decodeHeader(msg["Subject"])
emailBody = msg.get_payload()


orderId = emailSubject.split()[1]

soup = BeautifulSoup(emailBody, 'html.parser')



customerName = soup.find('span', class_='vm2-last_name')
customerPhone = soup.find('span', class_='vm2-phone_1')
customerEmail = soup.find('span', class_='vm2-email')
customerCity = soup.find('span', class_='vm2-virtuemart_state_id')
customerAddress = soup.find('span', class_='vm2-address_1')



In [ ]:


print( 'MessageId : ' + str(message_id) + '\n')    
print( 'Date : ' + emailDate + '\n')
print( 'From : ' + emailFrom  + '\n')
print( 'Subject : ' + emailSubject + '\n')
#print( 'Body : ' + emailBody + '\n')
print( 'OrderId : ' + orderId + '\n')
print( 'customerName : ' + (customerName.get_text() if (customerName) else '') + '\n')
print( 'customerPhone : ' + (customerPhone.get_text() if (customerPhone) else '') + '\n')
print( 'customerEmail : ' + (customerEmail.get_text() if (customerEmail) else '') + '\n')
print( 'customerCity : ' + (customerCity.get_text() if (customerCity) else '') + '\n')
print( 'customerAddress : ' + (customerAddress.get_text() if (customerAddress) else '') + '\n')

In [ ]:
class EmailMessage:
    messageId